In [ ]:
import os
import sys
sys.path.append("..")

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

import CST as CST
import utils as utils
from load_data import load_aj_idc

tf.enable_eager_execution() 

In [ ]:
dist_params = {
    1: {
            "contrast": {"lower": 0.8, "upper": 1.2},
            "color": {"factor": [5,5,5]},
            "blur": {"kernel_size": 2, "sigma": 0.},  # kernel size is 'kernel_size * 2 + 1'
            "brightness": {"max_delta":0.3}
    },
    2: {
            "contrast": {"lower": 0.8, "upper": 1.2},
            "color": {"factor": [5,5,5]},
            "blur": {"kernel_size": 2, "sigma": 1.},  # kernel size is 'kernel_size * 2 + 1'
            "brightness": {"max_delta":0.3}
    },
    3: {
            "contrast": {"lower": 0.6, "upper": 1.6},
            "color": {"factor": [20,20,20]},
            "blur": {"kernel_size": 2, "sigma": 5.},  # kernel size is 'kernel_size * 2 + 1'
            "brightness": {"max_delta":0.5}
    },
    4: {
            "contrast": {"lower": 0.6, "upper": 1.6},
            "color": {"factor": [20,20,20]},
            "blur": {"kernel_size": 4, "sigma": 5.},  # kernel size is 'kernel_size * 2 + 1'
            "brightness": {"max_delta":0.5}
    }
}

In [ ]:
# generator parameters
data_path = "../data/aj/IDC_regular_ps50_idx5"
batch_size = 64
tile_size = 128


# Download the data if the folder does not exist
if not os.path.exists(data_path):
    pathlib.Path("../data/aj").mkdir(parents=True, exist_ok=True)
    load_aj_idc("../data/aj")


gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=utils.normalize_image
)

flow = gen.flow_from_directory(
    directory=data_path,
    target_size=(tile_size,tile_size),
    color_mode='rgb',  # rgb for color
    batch_size=batch_size,
)


In [ ]:
def visualize_augmentations(num_images):
    # Sample a batch from a dataset
    batch = next(iter(flow))
    if type(batch) == tuple:
        images = batch[0][:num_images]
    else:
        images = batch[:num_images]
    
    # Apply augmentations
    augmented_images = zip(
        utils.denormalize_image(images).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[1]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[1]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[1]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[1]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[2]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[2]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[2]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[2]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[3]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[3]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[3]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[3]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[4]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[4]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[4]).numpy()).astype(int),
        utils.denormalize_image(CST.dist_fn(images, tile_size, dist_params[4]).numpy()).astype(int),
    )
    
    row_titles = [
        "Original:",
        "DC1:", "", "", "",
        "DC2:", "", "", "",
        "DC3:", "", "", "",
        "DC4:", "", "", "",
    ]
    plt.figure(figsize=(num_images * 2.2, 17 * 2.2), dpi=100)
    for column, image_row in enumerate(augmented_images):
        for row, image in enumerate(image_row):
            plt.subplot(17, num_images, row * num_images + column + 1)
            plt.imshow(image)
            if column == 0:
                plt.title(row_titles[row], loc="left", fontsize=20)
            plt.axis("off")
    plt.tight_layout()
    
visualize_augmentations(num_images=8)